# Working with Resource Permissions

### Introduction

Now so far we have seen two types of policies.

* We have primarily worked with one type of policy, **identity** policies, which attach to our IAM role.  We have seen that the identity policy is used to attach permissions to a role, and that the role acts like a keycard we can give to resources.  So far our identity policy has granted ECR access.

* We also saw the **assume role policy** (or the trust policy), which allows our ec2 machine to machine to assume the role (ie. hold onto the keycard).  

```bash
resource "aws_iam_role" "ec2_ecr_role" {
    name = "ec2-ecr-allow"

    assume_role_policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
      Action = "sts:AssumeRole"
      Effect = "Allow"
      Principal = {
      Service = "ec2.amazonaws.com"
      }
      ...
```

Below, you can see the two policies we attached -- the Trust Policy (assume role) and the Identity policy granting ECR access.

<img src="./ec2-ecr-permissions.png" width="60%">

Now the last policy to learn about is a **resource policy**, and that's what we'll learn about here.

### Adding Resource Permissions

A resource policy operates slightly differently than an identity policy.  If an IAM policy  acts like a keycard, granting access to whoever holds the keycard, then a resource policy is like a bouncer, standing in front of our ECR (or another) resource, saying who can enter.

This means that our resource policy is attached directly to the resource, instead of being attached to the role.

We can see this by looking at the `aws_ecr_repository_policy` defined in the `tf-resource/resource_ec2.tf` file.  There we'll see the following:

```bash
resource "aws_ecr_repository_policy" "ec2_access" {
  repository = "app-repo"

  policy = jsonencode({
    Version = "2012-10-17",
    Statement = [
      {
        Sid    = "AllowEC2RoleAccess",
        Effect = "Allow",
        Principal = {
          AWS = aws_iam_role.ec2_resource_access_role.arn
        },
        Action = [
          "ecr:GetDownloadUrlForLayer",
          "ecr:BatchGetImage",
          "ecr:BatchCheckLayerAvailability"
        ]
      }
    ]
  })
}
```

Look at the second line of the statement above:
    
```bash
repository = "app-repo"
```

    
So you can see that the resource policy attaches directly to the repository named `app-repo` -- it does not attach to an IAM role.  And then by attaching to that `app-repo`, it specifies who can access the repo.  And there, it says the `Principal` -- the person who can walk through the door can be the those who hold our `aws_iam_role` keycard.

resource "aws_iam_role"

### Applying our changes

So now navigate to the `tf-resource` folder, and the `resource_ec2.tf` file.  There you'll see we have defined an iam_role up top, followed by an instance profile that we'll attach to an ec2 machine, and then comes the `ecr_repository_policy`.  

Update the `repository=app-repo`, to match one of your repositories.  

You can find it by navigating to `Amazon ECR > Private Registry > Repositiories`.

<img src="./existing-repos.png" width="60%">

So we'll specify that we want to give access to this repository, and then later down we specify the principal indicating who gets access to this repository.
```bash
Principal = {
          AWS = aws_iam_role.ec2_resource_access_role.arn
        }
```

> So again, you can think of this being like a bouncer who now has certain resources on it's list it will allow in. 

And then later on you can see that we attached that role above to an EC2 instance.

```bash
resource "aws_instance" "ec2_resource" {
  ami           = "ami-0d7a109bf30624c99"
  instance_type = "t2.micro"
  iam_instance_profile = aws_iam_instance_profile.ec2_resource_profile.name
```

For the `aws_instance`, you'll need to update the `key_name` property, and potentially the `iam` property.

Ok, so with those changes let's apply this code.

```bash
terraform init
terraform apply
```

1. Checking the resource ec2

So, now if you look at what we outputted to the terminal, you should see the instructions to ssh into the ec2 resource.

So confirm that you can list the images with the following.

> * `aws ecr list-images --repository-name=app-repo`

In [ ]:
* Seeing the permissions

Ok, so remember, so far we grant this permissions directly on the resource -- as opposed to the IAM.  

You can see this, by going to the relevant repository (for me, it's `app-repo`), and then clicking on `Repo-Name > Permissions`.

<img src="./app-repo-permissions.png" width="60%">

So above, you can see that the resource gives allow access to those with `ec2-access-role` various actions, like Describe Repository.

And that's why we were able to describe the repo from our EC2 image.

* One gotcha

Unfortunately, if from our EC2 we try the following:

> ```bash
aws ecr get-login-password --region us-east-1 | sudo docker login --username AWS --password-stdin **********.dkr.ecr.us-east-1.amazonaws.com
```

You may get an error.  Unfortunately, AWS does not allow granting `ecr:GetAuthorizationToken` from a resource policy.  We'll have to use an IAM policy to get the authorization token.

> You don't have to memorize this -- it's just to point out some of the fun things that can go wrong.

### 2. Adding an Identity Policy

So what if we add on another machine that does not have access through the resource policy, but through our good old identity policy.  How does a resource policy interact with that?

So go to the `tf-resource/iam_ec2.tf` file.  You'll see that part of it is commented out, so now uncommented so that nothing is commented.

Then in the ec2 instance: 

* update the `key_name` attribute and potentially the `iam` value.  

Then run the following:

```bash
terraform apply
```

And ssh into the ec2 machine.

Ok, so the iam policy defined for the ec2 that we just created grants a generic `AmazonEC2ContainerRegistryReadOnly` policy.  Now that we have a resource policy on our repository, does the identity policy still work?

So you'll see instructions how to ssh into this machine:

```
ssh with into ec2_iam with the following
```

So ssh into the machine, and then try to access the repository.

```bash
aws ecr get-login-password --region us-east-1 | sudo docker login --username AWS --password-stdin *******.dkr.ecr.us-east-1.amazonaws.com

sudo docker pull *********.dkr.ecr.us-east-1.amazonaws.com/flask_app:latest
```

This works.

```bash
aws ecr list-images --repository-name=app-repo
```

This also works.


OK, so we just learned that, even though we have granted permissions to a role through the resource policy, we can also grant permissions to another role through the identity policy.  

* Clean up 

Run `terraform destroy` to remove your resources.

### Ok so where's the bouncer?

So now so far we have allowed access with our resource policy, but what if we wanted to deny access with that policy.

> **Note:** This next part you do not need to apply.  You can just read along.

Ok, so if you look at the policy below, you can see that again it is a `aws_ecr_resource_policy`, attached directly to the resource.  But this time we specify `Effect = Deny`, and denying the actions listed.  You'll also see the `Condition`, which we'll discuss below.

```bash
resource "aws_ecr_repository_policy" "ec2_deny" {
  repository = "app-repo"
  policy = jsonencode({
    Version = "2012-10-17",
    Statement = [{
        Sid    = "DenyECRAccess",
        Effect = "Deny",
        Condition = {
                "StringNotEquals": {
                    "aws:userid": ["admin", aws_iam_role.ec2_resource_access_role.name]
                }
            }
        Action = [
           "ecr:GetDownloadUrlForLayer",
          "ecr:BatchGetImage",
          "ecr:BatchCheckLayerAvailability",
          "ecr:GetAuthorizationToken"
        ]
      }]
}
```

Ok, so our condition is specifying the roles where we do want to give some access.  And that is those who we do not want to apply our deny access to.  And those roles are our admin, and the ec2 resource we want to access the ecr.

```bash
Condition = {
    "StringNotEquals": {
        "aws:userid": ["admin", aws_iam_role.ec2_resource_access_role.name]
    }
}
```

So we want the `ec2_resource` to not be denied because well, we want it to pull down our image from the repo.  And we want the `admin` to not be denied because terraform will need access to the repo to say change the permissions on the repo, or make any other changes.  If we forget to list the admin, then if we ever use terraform to try change something about our ECR repo, we'll see an error specifying that our admin user does not have access.

> And, in case your wondering, that deny access at the resource level will win out, even if we have an allow access through an IAM policy.  This is because AWS explicit deny will win out over an explicit allow. 

### Summary

In this lesson, we saw that, with with a resource policy, we can attach a policy directly to a resource.

```bash
resource "aws_ecr_repository_policy" "ec2_access" {
  repository = "app-repo"

  policy = jsonencode({
    Version = "2012-10-17",
    Statement = [
      {
        Sid    = "AllowEC2RoleAccess",
        Effect = "Allow",
        Principal = {
          AWS = aws_iam_role.ec2_resource_access_role.arn
        },
        Action = [
          "ecr:GetDownloadUrlForLayer",
          "ecr:BatchGetImage",
          "ecr:BatchCheckLayerAvailability"
        ]
      }
    ]
  })
}
```

So above, **the resource** is granting permissions to the specified role to perform the above actions on the resource.  And we can see this by navigating to the resource, and clicking on permissions.

<img src="./app-repo-permissions.png" width="60%">

Finally, we saw granting allow access to a role through a resource policy, does not remove our access through an identity policy.

However, if we specify a resource policy of deny, that will win out over any allow policies granted through an identity policy.

### Resources

[Repo Policy Examples](https://docs.aws.amazon.com/AmazonECR/latest/userguide/repository-policy-examples.html)